In [1]:
import numpy as np
from utils.bert import Load_RNABert_Model
import torch

In [2]:
model = Load_RNABert_Model('RNABERT_.pth')

device:  cuda
---Loaded---


In [ ]:
model.predict_embedding('AUGC').shape

In [ ]:
torch.save(model.model.module.state_dict(), './RNABERT_.pth')

In [ ]:
# python MLM_SFP.py --pretraining bert_mul_2.pth --data_embedding sample/aln/sample.raw.fa --embedding_output ./test.txt --batch 40